# Remodelando Dados

## Sobre os dados
Neste notebook, usaremos dados de temperatura diária do [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). Utilizaremos o conjunto de dados Global Historical Climatology Network - Daily (GHCND); veja a documentação [aqui](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf).

Esses dados foram coletados para a cidade de Nova York em outubro de 2018, usando a estação Boonton 1 (GHCND:USC00280907). Contém:
- a temperatura mínima diária (`TMIN`)
- a temperatura máxima diária (`TMAX`)
- a temperatura diária no momento da observação (`TOBS`)

*Nota: O NCEI faz parte da National Oceanic and Atmospheric Administration (NOAA) e, como pode ser visto na URL da API, esse recurso foi criado quando o NCEI era chamado de NCDC. Caso a URL deste recurso mude no futuro, você pode procurar por "NCEI weather API" para encontrar a versão atualizada.*

## Configuração
Precisamos importar o `pandas` e ler os dados no formato longo para começar:

In [1]:
import pandas as pd

long_df = pd.read_csv(
    'data/long_data.csv', usecols=['date', 'datatype', 'value']
).rename(
    columns={'value': 'temp_C'}
).assign(
    date=lambda x: pd.to_datetime(x.date),
    temp_F=lambda x: (x.temp_C * 9/5) + 32
)
long_df.head()

,datatype,date,temp_C,temp_F
0,TMAX,2018-10-01,21.1,69.98
1,TMIN,2018-10-01,8.9,48.02
2,TOBS,2018-10-01,13.9,57.02
3,TMAX,2018-10-02,23.9,75.02
4,TMIN,2018-10-02,13.9,57.02


## Transposição
A transposição troca as linhas pelas colunas. Usamos o atributo `T` para fazer isso:

In [2]:
long_df.set_index('date').head(6).T

date,2018-10-01,2018-10-01,2018-10-01,2018-10-02,2018-10-02,2018-10-02
datatype,TMAX,TMIN,TOBS,TMAX,TMIN,TOBS
temp_C,21.1,8.9,13.9,23.9,13.9,17.2
temp_F,69.98,48.02,57.02,75.02,57.02,62.96


## Pivotagem
Transformando do formato longo para o formato largo.

### `pivot()`
Podemos reestruturar nossos dados escolhendo uma coluna para ir no índice (`index`), uma coluna cujos valores únicos se tornarão nomes de colunas (`columns`), e os valores a serem colocados nessas colunas (`values`). O método `pivot()` pode ser usado quando não precisamos realizar nenhuma agregação além da reestruturação (quando nosso índice é único); se este não for o caso, precisamos do método `pivot_table()`, que será abordado no capítulo 4 do livro.

In [3]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values='temp_C'
)
pivoted_df.head()

datatype,TMAX,TMIN,TOBS
date,,,
2018-10-01,21.1,8.9,13.9
2018-10-02,23.9,13.9,17.2
2018-10-03,25.0,15.6,16.1
2018-10-04,22.8,11.7,11.7
2018-10-05,23.3,11.7,18.9


Agora que os dados foram pivotados, temos um formato largo que nos permite obter estatísticas resumidas:

In [4]:
pivoted_df.describe()

datatype,TMAX,TMIN,TOBS
count,31.000000,31.000000,31.000000
mean,16.829032,7.561290,10.022581
std,5.714962,6.513252,6.596550
min,7.800000,-1.100000,-1.100000
25%,12.750000,2.500000,5.550000
50%,16.100000,6.700000,8.300000
75%,21.950000,13.600000,16.100000
max,26.700000,17.800000,21.700000


Também podemos fornecer múltiplos valores para pivotar, o que resultará em um índice hierárquico:

In [5]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values=['temp_C', 'temp_F']
)
pivoted_df.head()

temp_C             temp_F              
datatype     TMAX  TMIN  TOBS   TMAX   TMIN   TOBS
date                                              
2018-10-01   21.1   8.9  13.9  69.98  48.02  57.02
2018-10-02   23.9  13.9  17.2  75.02  57.02  62.96
2018-10-03   25.0  15.6  16.1  77.00  60.08  60.98
2018-10-04   22.8  11.7  11.7  73.04  53.06  53.06
2018-10-05   23.3  11.7  18.9  73.94  53.06  66.02

Com o índice hierárquico, se quisermos selecionar `TMIN` em Fahrenheit, precisaremos primeiro selecionar `temp_F` e depois `TMIN`:

In [6]:
pivoted_df['temp_F']['TMIN'].head()

date
2018-10-01    48.02
2018-10-02    57.02
2018-10-03    60.08
2018-10-04    53.06
2018-10-05    53.06
Name: TMIN, dtype: float64

### `unstack()`

Temos trabalhado com um único índice ao longo deste capítulo; no entanto, podemos criar um índice a partir de qualquer número de colunas com `set_index()`. Isso nos dá um índice do tipo `MultiIndex`, onde o nível mais externo corresponde ao primeiro elemento da lista fornecida ao `set_index()`:

In [7]:
multi_index_df = long_df.set_index(['date', 'datatype'])
multi_index_df.head().index

MultiIndex([('2018-10-01', 'TMAX'),
            ('2018-10-01', 'TMIN'),
            ('2018-10-01', 'TOBS'),
            ('2018-10-02', 'TMAX'),
            ('2018-10-02', 'TMIN')],
           names=['date', 'datatype'])

Note que agora existem 2 seções de índice no dataframe:

In [8]:
multi_index_df.head()

temp_C  temp_F
date       datatype                
2018-10-01 TMAX        21.1   69.98
           TMIN         8.9   48.02
           TOBS        13.9   57.02
2018-10-02 TMAX        23.9   75.02
           TMIN        13.9   57.02

Com um índice do tipo `MultiIndex`, não podemos mais usar `pivot()`. Agora devemos usar `unstack()`, que por padrão move o índice mais interno para as colunas:

In [9]:
unstacked_df = multi_index_df.unstack()
unstacked_df.head()

temp_C             temp_F              
datatype     TMAX  TMIN  TOBS   TMAX   TMIN   TOBS
date                                              
2018-10-01   21.1   8.9  13.9  69.98  48.02  57.02
2018-10-02   23.9  13.9  17.2  75.02  57.02  62.96
2018-10-03   25.0  15.6  16.1  77.00  60.08  60.98
2018-10-04   22.8  11.7  11.7  73.04  53.06  53.06
2018-10-05   23.3  11.7  18.9  73.94  53.06  66.02

## Melting

Transformando do formato largo para o formato longo.

### Configuração

In [11]:
wide_df = pd.read_csv('data/wide_data.csv')
wide_df.head()

,date,TMAX,TMIN,TOBS
0,2018-10-01,21.1,8.9,13.9
1,2018-10-02,23.9,13.9,17.2
2,2018-10-03,25.0,15.6,16.1
3,2018-10-04,22.8,11.7,11.7
4,2018-10-05,23.3,11.7,18.9


### `melt()`

Para transformar do formato largo para o formato longo, usamos o método `melt()`. Precisamos especificar:

- `id_vars`: qual coluna ou colunas identificam exclusivamente uma linha no formato largo (`date`, neste caso)
- `value_vars`: a coluna ou colunas que contêm os valores (`TMAX`, `TMIN` e `TOBS`, aqui)

Opcionalmente, também podemos fornecer:

- `value_name`: como chamar a coluna que conterá todos os valores após o derretimento
- `var_name`: como chamar a coluna que conterá os nomes das variáveis medidas

In [14]:
melted_df = wide_df.melt(
    id_vars='date',
    value_vars=['TMAX', 'TMIN', 'TOBS'],
    value_name='temp_C',
    var_name='measurement'
)
melted_df.head()

,date,measurement,temp_C
0,2018-10-01,TMAX,21.1
1,2018-10-02,TMAX,23.9
2,2018-10-03,TMAX,25.0
3,2018-10-04,TMAX,22.8
4,2018-10-05,TMAX,23.3


### `stack()`

Outra opção é `stack()`, que irá pivotar as colunas do dataframe para o nível mais interno do índice (resultando em um índice do tipo `MultiIndex`). Para ilustrar isso, vamos definir nosso índice como a coluna `date`:

In [15]:
wide_df.set_index('date', inplace=True)
wide_df.head()

,TMAX,TMIN,TOBS
date,,,
2018-10-01,21.1,8.9,13.9
2018-10-02,23.9,13.9,17.2
2018-10-03,25.0,15.6,16.1
2018-10-04,22.8,11.7,11.7
2018-10-05,23.3,11.7,18.9


Ao executar `stack()` agora, criaremos um segundo nível em nosso índice que conterá os nomes das colunas do nosso dataframe (`TMAX`, `TMIN`, `TOBS`). Isso nos deixará com um objeto `Series` contendo os valores:

In [16]:
stacked_series = wide_df.stack()
stacked_series.head()

date            
2018-10-01  TMAX    21.1
            TMIN     8.9
            TOBS    13.9
2018-10-02  TMAX    23.9
            TMIN    13.9
dtype: float64

Podemos usar o método `to_frame()` em nosso objeto `Series` para transformá-lo em um objeto `DataFrame`. Como a série não tem um nome no momento, passaremos o nome como um argumento:

In [17]:
stacked_df = stacked_series.to_frame('values')
stacked_df.head()

values
date                   
2018-10-01 TMAX    21.1
           TMIN     8.9
           TOBS    13.9
2018-10-02 TMAX    23.9
           TMIN    13.9

Mais uma vez, temos um índice do tipo `MultiIndex`:

In [18]:
stacked_df.head().index

MultiIndex([('2018-10-01', 'TMAX'),
            ('2018-10-01', 'TMIN'),
            ('2018-10-01', 'TOBS'),
            ('2018-10-02', 'TMAX'),
            ('2018-10-02', 'TMIN')],
           names=['date', None])

Infelizmente, não temos um nome para o nível `datatype`:

In [19]:
stacked_df.index.names

FrozenList(['date', None])

Podemos usar `set_names()` para resolver isso:

In [20]:
stacked_df.index.set_names(['date', 'datatype'], inplace=True)
stacked_df.index.names

FrozenList(['date', 'datatype'])

<hr>
<div>
    <a href="./3-cleaning_data.ipynb">
        <button>&#8592; Previous Notebook</button>
    </a>
    <a href="./5-handling_data_issues.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>